**<font size = 5 > 0. SETUP LIBRARIES AND DEFINE FUNCTIONS** </font>

In [1]:
import pandas as pd
import numpy as np
import json
import csv
from ast import literal_eval
from datetime import datetime
from ruamel.yaml import YAML
import re

def remove_emo(str1):
    text = emoji_pattern.sub(r'', str1)
    return text

def remove_al(text):
    final = text.replace('ن','')
    return final

emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F" 
                           "\U0001F300-\U0001F5FF"   
                           "\U0001F680-\U0001F6FF"   
                           "\U0001F1E0-\U0001F1FF"   
                           u"\U0001F1F2-\U0001F1F4"   
                           "\U0001F1E6-\U0001F1FF"   
                           "\U0001F600-\U0001F64F"u"\U00002702-\U000027B0"u"\U000024C2-\U0001F251"u"\U0001f926-\U0001f937"u"\U0001F1F2"u"\U0001F1F4"u"\U0001F620"u"\u200d"u"\u2640-\u2642""]+", flags=re.UNICODE)


**<font size = 5 > 1. IMPORT CONFIG_FILE AND TWEETS** </font>

In [3]:
#####--- 1.0 Import Config_File
yaml        = YAML(typ='safe')
with open('/Users/ivanmera/Documents/dataman/input_data/Config_File_ospiti_1.yml', 'r') as fil:
        config = yaml.load(fil)
qr_general = config["quarta_rep1"]
nomi_veri = qr_general["quarta_rep_real"]

#####--- 1.1 Import tweets
tweets = []
for line in open('/Users/ivanmera/Documents/dataman/stream_json/quartarepubblica_pulito_1.json',
                 'r',
                 encoding="utf-8"):
    '''appendo ogni tweet in una lista'''
    tweets.append(json.loads(line))
    
#####--- 1.2 Write CSV    
with open("/Users/ivanmera/Documents/dataman/stream_csv/quartarepubblica_tweets_1.csv", "w", newline="", encoding='utf-8') as c:
    writer = csv.writer(c)
    '''definisco gli attributi del mio df
    '''
    writer.writerow(
        ["_id", "user_id", "user_name", "created_at", "text", "user_men","nome_pgr"])
    for tweet in tweets:
        if isinstance(tweet["_id"], dict): 
            _id = tweet.get('_id', {}).get('$oid')
        else:
            _id = tweet.get('_id',{})
        '''valuto se l'attributo user_id ha come valore un dict anzichè un numero
        '''
        if isinstance(tweet["user_id"], dict):
            user_id_ = tweet.get('user_id', {}).get('$numberLong')
        else:
            '''nel caso non lo fosse prendo direttamente il numero
            '''
            user_id_ = tweet.get('user_id', {})
        ''' prendo l'orario
        '''
        created_at_ = tweet.get('created_at', {}).get('$date')
        '''prendo username'''
        user_name = tweet.get('username', {})
        '''inserisco tutti i valori
        '''
        writer.writerow([_id,
            user_id_, user_name, created_at_, tweet["text"],
            tweet["user_mentions_name"],tweet["nome_pgr"]
        ])
c.close()

#####--- 1.3 Import CSV
df = pd.read_csv("/Users/ivanmera/Documents/dataman/stream_csv/quartarepubblica_tweets_1.csv") 

**<font size = 5 > 2. PROCESSING** </font>

In [7]:
#####--- 2.0 Set datetime format
df["created_at"] = pd.to_datetime(df["created_at"], format = "%Y-%m-%dT%H:%M:%SZ")

#####--- 2.1 Explode mentions and normalize
df["user_men"] = df["user_men"].apply(literal_eval)
df = df.explode("user_men")
df["user_men_normalize"] = df["user_men"].str.lower()
df["user_men_normalize"] = df["user_men_normalize"].astype(str)
df["user_men_normalize"] = df["user_men_normalize"].str.strip()
df["user_men_normalize_emo"] = df["user_men_normalize"].apply(remove_emo) # remove emoticon
df["user_men_normalize_emo"] = df["user_men_normalize_emo"].apply(remove_al)# demelonizzazione

#####--- 2.2 Filter mentions
df_final = df.loc[df["user_men_normalize_emo"].isin(nomi_veri)]
df_final["user_men_normalize_emo"] = df_final["user_men_normalize_emo"].str.title()
df_final["user_men_normalize_emo"] = df_final["user_men_normalize_emo"].str.replace("Dimartedì", "Di martedì")

#####--- 2.3 Export 
df_final.to_excel('/Users/ivanmera/Documents/dataman/stream_xlsx/quartarepubblica_pulito_1.xlsx', 
                  index = False)

In [18]:
df_final.head()

,_id,user_id,user_name,created_at,text,user_men,nome_pgr,user_men_normalize,user_men_normalize_emo
1518,5ffcb329bab666f05f083f8a,424084271,L'Italia Vera 🇮🇹,2021-01-11 21:20:52,RT @Capezzone: +++Appuntamento in tv 📺 +++\nSt...,Daniele Capezzone,Quarta repubblica,daniele capezzone,Daniele Capezzone
1518,5ffcb329bab666f05f083f8a,424084271,L'Italia Vera 🇮🇹,2021-01-11 21:20:52,RT @Capezzone: +++Appuntamento in tv 📺 +++\nSt...,Quarta Repubblica,Quarta repubblica,quarta repubblica,Quarta Repubblica
2719,5ffcb352bab666f05f084442,467573735,IL CITTADINO,2021-01-11 21:21:33,RT @MG58steelclass: @NicolaPorro @francetomm @...,Nicola Porro,Quarta repubblica,nicola porro,Nicola Porro
2811,5ffcb355bab666f05f08449f,1209530971617402880,Gerardo,2021-01-11 21:21:36,@NicolaPorro Che vada A fare il pastore che è ...,Nicola Porro,Quarta repubblica,nicola porro,Nicola Porro
3516,5ffcb36dbab666f05f084766,141529750,Laura Carrese ✖,2021-01-11 21:22:00,RT @VittorioSgarbi: A 90 anni dalla sua morte ...,Vittorio Sgarbi,Quarta repubblica,vittorio sgarbi,Vittorio Sgarbi
